# Run 2.1 with just CO2, CH4, N2O for now

In [ ]:
from fair import FAIR

from tqdm.autonotebook import tqdm
import numpy as np
import pandas as pd
import copy

In [ ]:
n_scen = 100  # 10000 in full run

In [ ]:
species_ids = {}
species_ids['co2_ffi'] = SpeciesID('CO2 Fossil fuel and industrial', Category.CO2_FFI, RunMode.EMISSIONS)
species_ids['co2_afolu'] = SpeciesID('CO2 AFOLU', Category.CO2_AFOLU, RunMode.EMISSIONS)
species_ids['co2'] = SpeciesID('CO2', Category.CO2, RunMode.FROM_OTHER_SPECIES)
species_ids['ch4'] = SpeciesID('CH4', Category.CH4, RunMode.EMISSIONS)
species_ids['n2o'] = SpeciesID('N2O', Category.N2O, RunMode.EMISSIONS)

In [ ]:
species = {}

co2 = np.zeros((551, n_scen))

for sample in tqdm(range(1, n_scen+1)):
    df = pd.read_csv('../data_processed/emissions_files/emissions%05d.csv' % sample, index_col=0)
    co2[:, sample-1] = df['CO2'].values
    # remember the order!
    species[sample] = [
        Species(species_ids['co2_ffi'], emissions=df['CO2'].values),
        Species(species_ids['co2_afolu'], emissions=np.zeros(551)),
        Species(species_ids['co2']),
        Species(species_ids['ch4'], emissions=df['CH4'].values),
        Species(species_ids['n2o'], emissions=df['N2O'].values),
    ]

In [ ]:
pl.plot(np.arange(1750, 2301), np.median(co2, axis=1))

In [ ]:
species[1][3]

## Defining Scenarios

We have declared all of our `Species`, and we now need to formally assign them to `Scenario`s in the code.

`scenarios` go into `fair` as a list, and we can take a peek at what our `Scenario` actually looks like to FaIR.

In [ ]:
scenarios = []
for scenario in tqdm(range(1, n_scen+1)):
    scenarios.append(Scenario(scenario, species[scenario]))

## The Configs

`Scenario`s define what goes into the model, but `Config`s define the model behaviour. There are two types of configs (well, actually three if you include the overall run configs, but these are defined at the top level and we'll revisit these later):

1. `ClimateResponse` defines the climate sensitivity, ocean heat capacities, heat transfer coefficient, etc. It defines how forcing is converted to temperature in the model. Only one `ClimateResponse` per config should be defined.
2. `SpeciesConfig` defines things such as greenhouse gas lifetimes, radiative forcing efficacy, gas cycle airborne fraction, and many other things. It has a lot of options, and a lot of flexibility. A list containing a `SpeciesConfig` for each defined species in the scenarios needs to be provided.

`Config`s are defined as follows:

```
config = Config(name, climate_response, species_configs)
```

We then provide a list of configs to FaIR.

In this example, we'll provide three configs representing high, medium and low climate sensitivity climate models, and also vary the `SpeciesConfig` in each to provide some variation in the carbon cycle feedbacks and the strength of the aerosol forcing.

### Climate response

We'll start with the climate response which is slightly easier. This is an $n$-box energy balance model, where $n=3$ by default (this can be changed in the `RunConfig`). 

In [ ]:
accept = np.loadtxt('../../data/ar6_ensemble_batches/final.csv', dtype=int)

In [ ]:
co2_pi = pd.read_csv('../../data/parameter_sets/co2_concentration_1750.csv').loc[accept]
co2_pi

In [ ]:
cc = pd.read_csv('../../data/parameter_sets/carbon_cycle.csv').loc[accept]
cc

In [ ]:
cr = pd.read_csv('../../data/parameter_sets/climate_response.csv').loc[accept]
cr

In [ ]:
calibrated_f4co2_mean = cr['F_4xCO2'].loc[accept].mean()
calibrated_f4co2_mean

In [ ]:
scale = pd.read_csv('../../data/parameter_sets/forcing_scaling.csv').loc[accept]
scale

In [ ]:
cr['c1']

In [ ]:
climate_response = {}
for config_label in range(1001):
    climate_response[config_label] = ClimateResponse(
        ocean_heat_capacity = np.array([cr['c1'].iloc[config_label], cr['c2'].iloc[config_label], cr['c3'].iloc[config_label]]),
        ocean_heat_transfer = np.array([cr['kappa1'].iloc[config_label], cr['kappa2'].iloc[config_label], cr['kappa3'].iloc[config_label]]),
        deep_ocean_efficacy = cr['epsilon'].iloc[config_label],
        stochastic_run = True, #False
        sigma_eta = cr['sigma_eta'].iloc[config_label],
        sigma_xi = cr['sigma_xi'].iloc[config_label],
        seed = config_label * 780 + 13
    )

### Species configuration

There are a lot of options - but only a few per species are used. And to make things easier, the `default_species_config` module contains many really useful defaults.

In [ ]:
species_ids

<div class="alert alert-block alert-warning">
It's really important to use `copy` here, otherwise we are just creating multiple references to the default dict.
</div>

In [ ]:
species_configs = {}

for config_label in range(1001):
    species_configs[config_label] = []
    for species_label in ['co2_ffi', 'co2_afolu', 'co2', 'ch4', 'n2o']:
        # use copy here!
        species_configs[config_label].append(copy.copy(default_species_config[species_label]))
        # the defaults also contain the SpeciesID that has a default name and run mode. We want to override this.
        species_configs[config_label][-1].species_id = species_ids[species_label]

#### Modify carbon cycle configs

In small problems like this you might be able to remember the indices of each specie, but in case you need a reminder - and this is very useful for big problems such as a full CMIP6 run:

In [ ]:
for ispecconf in range(len(species_configs[config_label])):
    print (ispecconf, species_configs[config_label][ispecconf].species_id.name)

So if we want to modify the carbon cycle, we have to modify the properties in `SpeciesConfig` of CO2, which is list element number 2.

First, let's examine the defaults:

In [ ]:
for config_label in range(1001):
    # Carbon cycle
    species_configs[config_label][2].iirf_0 = cc['r0'].iloc[config_label]
    species_configs[config_label][2].iirf_airborne = cc['rA'].iloc[config_label]
    species_configs[config_label][2].iirf_cumulative = cc['rC'].iloc[config_label]
    species_configs[config_label][2].iirf_temperature = cc['rT'].iloc[config_label]
    species_configs[config_label][2].baseline_concentration = co2_pi['co2_concentration'].iloc[config_label]
    
    # Scale factors for forcing
    species_configs[config_label][2].scale = 1 + 0.561 * (calibrated_f4co2_mean - cr['F_4xCO2'].iloc[config_label])/calibrated_f4co2_mean
    species_configs[config_label][3].scale = scale['CH4'].iloc[config_label]
    species_configs[config_label][4].scale = scale['N2O'].iloc[config_label]

### now, let's assign to Configs

In [ ]:
configs = []
for config_label in range(1001):
    configs.append(Config(config_label, climate_response[config_label], species_configs[config_label]))

## Now, we can run FaIR

In [ ]:
from fair21 import FAIR

In [ ]:
timestep = 1  # not required, but demonstrated
fair = FAIR(scenarios, configs, timestep)

In [ ]:
fair.run(progress=True)

In [ ]:
fair.temperature.shape

In [ ]:
df_ssp_median=pd.read_csv('../../data/ar6_ensemble_batches/ssp_median_temperature.csv', index_col=0)

In [ ]:
pl.fill_between(
    np.arange(1750, 2301), 
    np.percentile(fair.temperature[:,:,:,0,0] - fair.temperature[100:151,:,:,0,0].mean(axis=0), 1, axis=(1,2)),
    np.percentile(fair.temperature[:,:,:,0,0] - fair.temperature[100:151,:,:,0,0].mean(axis=0), 99, axis=(1,2)),
    color='0.9'
)
pl.fill_between(
    np.arange(1750, 2301), 
    np.percentile(fair.temperature[:,:,:,0,0] - fair.temperature[100:151,:,:,0,0].mean(axis=0), 5, axis=(1,2)),
    np.percentile(fair.temperature[:,:,:,0,0] - fair.temperature[100:151,:,:,0,0].mean(axis=0), 95, axis=(1,2)),
    color='0.8'
)
pl.plot(
    np.arange(1750, 2301), 
    np.percentile(fair.temperature[:,:,:,0,0] - fair.temperature[100:151,:,:,0,0].mean(axis=0), 50, axis=(1,2)),
    color='k',
    label='RFF median'
)
pl.plot(
    np.arange(1750, 2301),
    df_ssp_median['ssp126'],
    color='blue',
    label='ssp126'
)
pl.plot(
    np.arange(1750, 2301),
    df_ssp_median['ssp245'],
    color='orange',
    label='ssp245'
)
pl.plot(
    np.arange(1750, 2301),
    df_ssp_median['ssp370'],
    color='red',
    label='ssp370'
)
pl.plot(
    np.arange(1750, 2301),
    df_ssp_median['ssp585'],
    color='purple',
    label='ssp585'
)

pl.title('RFF first 100 scenarios: temperature')
pl.xlabel('year')
pl.ylabel('Temperature anomaly (K)')
pl.legend(loc='upper left')
pl.xlim(2000, 2300)

In [ ]:
pl.fill_between(
    np.arange(1750, 2301), 
    np.percentile(fair.concentration_array[:,:,:,2,0], 5, axis=(1,2)),
    np.percentile(fair.concentration_array[:,:,:,2,0], 95, axis=(1,2)),
    color='0.8'
)
pl.plot(
    np.arange(1750, 2301), 
    np.percentile(fair.concentration_array[:,:,:,2,0], 50, axis=(1,2)),
    color='k'
)
pl.title('RFF first 100 scenarios: CO2 concentration')
pl.xlabel('year')
pl.ylabel('ppm')
pl.xlim(2000, 2300)